In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [ ]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [4]:
data = pd.read_excel('data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [5]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [7]:
x.drop('lat', axis = 1, inplace = True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

### RFE

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

Сначала RFE обучается на изначальной выборке, и происходит оценка важности каждого признака. Затем наименее важные фичи удаляются. Эта процедура рекурсивно повторяется на сокращённом наборе до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков в выборке.

Выделим три наиболее значимых признака:

In [10]:
from sklearn.feature_selection import RFE

In [11]:
y = data['price']
x = data.drop(columns='price')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [13]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Также узнаем, как RFE проранжировал все доступные признаки:

In [14]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [15]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

В качестве фильтров для выбора признаков используются уже знакомые нам статистики, такие как коэффициент корреляции Пирсона, ANOVA и т. д. При этом выбор статистических показателей сильно зависит от типов переменных в данных.

Чем больше известно о типе данных, тем проще выбрать подходящую статистическую меру для метода отбора признаков на основе фильтра. Ниже приведена схема-помощник в выборе метода селекции признаков.

In [16]:
from sklearn.feature_selection import SelectKBest, f_regression

Библиотека sklearn обеспечивает реализацию большинства полезных статистических показателей, например:

        коэффициента корреляции Пирсона: f_regression();
        дисперсионного анализа ANOVA: f_classif();
        хи-квадрата: chi2();
        взаимной информации: mutual_info_classif() и mutual_info_regression().

Кроме того, библиотека SciPy обеспечивает реализацию многих других статистических данных, таких как тау Кендалла (kendalltau) и ранговая корреляция Спирмена (spearmanr).

sklearn также предоставляет множество различных методов фильтрации после расчёта статистики для каждой входной переменной с целевой.

Два наиболее популярных метода:

        выбор k лучших переменных: SelectKBest;
        выбор переменных верхнего процентиля: SelectPercentile.



In [22]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

На этот раз odometer оказался в топе.

## Задание 9.5. Модуль ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. Загрузите полученный ноутбук (в формате IPYNB) в форму ниже.

Сначала выделим три столбца-признака, которые мы определили с помощью RFE и разделим датасет на тренировочный и тестовый.

In [32]:
y = data['price']
x = data[['year', 'cylinders', 'lat']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Теперь построим модель линейной регрессии, основанной на выделенных признаках и отобразим метрику.

In [33]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


Можно уже заметить, что метрика значительно ухудшилась по сравнению с моделью, которая основана на всех доступных признаках.

Теперь проведем аналогичные манипуляции для признаков, отобранных с помощью SelectKBest.

In [34]:
y = data['price']
x = data[['year', 'cylinders', 'odometer']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [35]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4708.946


Метрика улучшилась на 300 единиц. Таким образом метод фильтрации SelectKBest показал себя лучше в задаче. Однако, такой результат также не идеален. Модель, построенная на всех признаках, кроме 'lat', показывает лучшую метрику.